### Assignment 1:
* Build 2 layer network for Logistic Regression Classifier using pure python/numpy
  * Input data x : 20 x 784 (20 is the batch size)
  * label y : 20 x 10
  * Weight Variable W with random_normal initialization
  * Bias variable b with zeros
  * Use softmax activation => softmax(x.W+b)
  * Use OneHot encoding and MSE loss! Not the best way for classification but just for practice
  * Train the model on MNIST

In [ ]:
import os, struct
from array import array as pyarray
from numpy import  array, zeros
import numpy as np
import random  
import numpy as np 
import matplotlib.pyplot as plt


The first function is to load the MNIST dataset

In [ ]:
def load_mnist(dataset="training", digits=np.arange(10), path="."):
    """
    Loads MNIST files into numpy arrays 
    """

    if dataset == "training":
        fname_img = os.path.join('sample_data', 'train-images.idx3-ubyte')
        fname_lbl = os.path.join('sample_data', 'train-labels.idx1-ubyte')
    elif dataset == "testing":
        fname_img = os.path.join('sample_data', 't10k-images.idx3-ubyte')
        fname_lbl = os.path.join('sample_data', 't10k-labels.idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    flbl = open(fname_lbl, 'rb')
    magic_nr, size = struct.unpack(">II", flbl.read(8))
    lbl = pyarray("b", flbl.read())
    flbl.close()

    fimg = open(fname_img, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", fimg.read(16))
    img = pyarray("B", fimg.read())
    fimg.close()

    ind = [ k for k in range(size) if lbl[k] in digits ]
    len_of_index = len(ind)

    random.seed(30)
    #random number in the range of the dataset - to shufftle the dataset
    rand_num = random.randrange(len_of_index - 20)
    # 20 is the batch size
    images = zeros((20, rows, cols) )
    labels = zeros((20))
    for i in range(20):
        images[i] = array(img[ ind[rand_num+i]*rows*cols : (ind[rand_num+i]+1)*rows*cols ]).reshape((rows, cols))
        labels[i] = lbl[ind[rand_num+i]]

    return images, labels

2nd fuction to calculate the softmax function

In [ ]:
def softmax(z_matrix):
    number_of_samples,number_of_classes = z_matrix.shape
    expected_output = np.zeros(z_matrix.shape)
    
    for i in range(number_of_samples):    
        normalization_value = 0
        for j in range(number_of_classes):
            expected_output[i][j] = np.exp(z_matrix[i][j])
            normalization_value += expected_output[i][j] 
        expected_output[i] /= normalization_value
    return expected_output

3rd function to calculate the gradient descent for each iteration

In [ ]:
def Gradient_Descent_for_one_sample(actual_y,y_prime,input_x):
    number_of_classes = len(actual_y)
    dl_dy =  actual_y - y_prime
    dy_dz = np.zeros((number_of_classes,number_of_classes))
    
    for i in range(number_of_classes): 
        for j in range(number_of_classes):
            if(i == j):
                sij_value = 1
            else:
                sij_value = 0
            dy_dz[i,j] = y_prime[i] * (sij_value - y_prime[j])
            
    dz_dw = input_x.reshape((input_x.shape[0],1))
    gradient_b = dl_dy.reshape((1,number_of_classes))@dy_dz
    gradient_w = dz_dw@gradient_b
    return gradient_b,gradient_w

Then we will load the the Mnist dataset into arrays:
 x_train (for the features) and temp_y_train (as a temp array for the labels)

In [ ]:
# Read in training data as features and labels
X_train, temp_y_train = load_mnist('training') 
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_train = np.divide(X_train, 256)

Customizing the y_train array to fit into our classification problem, as the output should be list of probabilities of 10 classes
(so the position of the class where this label related to  = 1 and the positions related to other classes = 0)

In [ ]:
number_of_samples = 20
number_of_classes = 10

# change triainging and testing to fit softmax regression
y_train = np.zeros((number_of_samples,number_of_classes))

for i in range(20):
    y_train[i][int(temp_y_train[i])] = 1

Then the training step

In [ ]:
# Intializing the weight with random values drawn from a normal distribution with "mean = 0" & "sigma = 0.1"  
mu, sigma = 0, 0.1
np.random.seed(1)
W = np.random.normal(mu, sigma, (X_train.shape[1],10))
b = np.zeros((1,10))
number_of_epochs = 1000

for k in range(number_of_epochs):
    # feed forward
    #layer 0
    l0 = X_train
    # layer 1
    l1 = l0@W+b
    # Expected output
    expected_output = softmax(l1)
    
    delta_w = np.zeros(W.shape)
    delta_b = np.zeros(b.shape)
    
    # backpropagate - update weight
    for i in range (number_of_samples):
        d_b,d_w = Gradient_Descent_for_one_sample(y_train[i],expected_output[i],X_train[i])
        delta_w += d_w
        delta_b += d_b
        
    delta_w = (-2/number_of_samples) * delta_w
    delta_b = (-2/number_of_samples) * delta_b
    
    W -= delta_w
    b -= delta_b
  
for i in range (number_of_samples):
  print ("expect output",expected_output[i])
  print ("labels",y_train[i])

expect output [1.07937170e-04 9.88611770e-01 1.04859554e-03 6.29502249e-04
 5.04003334e-04 2.32558328e-04 2.89909455e-04 9.54906880e-04
 1.68314737e-03 5.93766978e-03]
labels [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
expect output [3.02587669e-04 3.66486443e-04 9.71760502e-04 3.33207083e-03
 9.80502057e-04 6.38416201e-04 7.98230839e-04 9.30314178e-04
 1.87370765e-03 9.89805924e-01]
labels [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
expect output [1.84458535e-03 5.85274253e-04 9.90849851e-01 7.38923553e-04
 7.74642448e-04 4.96561747e-05 7.02489918e-04 1.42000666e-03
 3.00988698e-03 2.46832068e-05]
labels [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
expect output [2.67795759e-04 4.43470422e-03 1.04355251e-03 1.59288916e-03
 8.59068100e-04 5.63235755e-04 6.32645087e-04 3.23858352e-03
 1.98400457e-03 9.85383521e-01]
labels [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
expect output [3.79214133e-04 3.52447809e-05 6.63762289e-04 9.92459219e-01
 1.26715791e-05 2.94856387e-05 1.21394306e-04 2.71503367e-04
 2.37384126e-03 3.65366343e-03]
labe